## 일일 박스오피스 데이터 & 영화정보 merge

- 응답구조
    - boxofficeType :문자열	박스오피스 종류
    - showRange : 	문자열	박스오피스 조회 일자를 출력
    - rnum	문자열	순번을 출력
    - rank	문자열	해당일자의 박스오피스 순위를 출력
    - rankInten	문자열	전일대비 순위의 증감분을 출력
    - rankOldAndNew	문자열	랭킹에 신규진입여부를 출력
    - “OLD” : 기존 , “NEW” : 신규
    - movieCd: 문자열	영화의 대표코드를 출력
    - movieNm : 문자열	영화명(국문)을 출력
    - openDt : 문자열	영화의 개봉일을 출력
    - salesAmt : 문자열	해당일의 매출액을 출력
    - salesShare : 문자열	해당일자 상영작의 매출총액 대비 해당 영화의 매출비율을 출력
    - salesInten : 문자열	전일 대비 매출액 증감분을 출력
    - salesChange : 문자열	전일 대비 매출액 증감 비율을 출력
    - salesAcc : 문자열	누적매출액을 출력
    - audiCnt : 문자열	해당일의 관객수를 출력
    - audiInten : 문자열	전일 대비 관객수 증감분을 출력
    - audiChange : 문자열	전일 대비 관객수 증감 비율을 출력
    - audiAcc : 문자열	누적관객수를 출력합니다.
    - scrnCnt : 문자열	해당일자에 상영한 스크린수를 출력
    - showCnt : 문자열	해당일자에 상영된 횟수를 출력

In [1]:
import pandas as pd
import numpy as np
import time

df3 = pd.read_csv('../data/boxoffice_y_24.csv')
df2 = pd.read_csv('../data/boxoffice_y_2023.csv')
df1 = pd.read_csv('../data/boxoffice_y_1519.csv')

In [2]:
concat_df = pd.concat([df1, df2, df3], axis = 0)

In [3]:
df = concat_df.copy()

In [4]:
df['current_date']

0      2015-01-01
1      2015-01-01
2      2015-01-01
3      2015-01-01
4      2015-01-01
          ...    
905    2024-03-31
906    2024-03-31
907    2024-03-31
908    2024-03-31
909    2024-03-31
Name: current_date, Length: 33780, dtype: object

In [21]:
df['current_date'] = pd.to_datetime(df['current_date'])
movie_codes = df[df['current_date'].dt.year >= 2024]['movieCd'].drop_duplicates().tolist()

### 영화 상세정보 크롤링

In [6]:
# import requests
# import pandas as pd

# key = 'b713496d3bb846528153ba2985029132'
# movieCd = 20182530
# url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
# response = requests.get(url)
# r_data = response.json()

In [7]:
import requests
import pandas as pd

key = 'b713496d3bb846528153ba2985029132'
movie_codes = df['movieCd'].drop_duplicates().tolist()[0:2]

info_dfs = []

for movieCd in movie_codes:
    url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
    response = requests.get(url)
    r_data = response.json()

    # 응답 선택, 값이 없으면 none
    selected_data = {
        'movieCd': r_data['movieInfoResult']['movieInfo'].get('movieCd', None),
        'movieNm': r_data['movieInfoResult']['movieInfo'].get('movieNm', None),
        'showTm': r_data['movieInfoResult']['movieInfo'].get('showTm', None),
        'prdtYear': r_data['movieInfoResult']['movieInfo'].get('prdtYear', None),
        'openDt': r_data['movieInfoResult']['movieInfo'].get('openDt', None),
        'prdtStatNm': r_data['movieInfoResult']['movieInfo'].get('prdtStatNm', None),
        'typeNm': r_data['movieInfoResult']['movieInfo'].get('typeNm', None),
        'nationNm': r_data['movieInfoResult']['movieInfo']['nations'][0].get('nationNm', None),
        'genreNm': r_data['movieInfoResult']['movieInfo']['genres'][0].get('genreNm', None) if r_data['movieInfoResult']['movieInfo']['genres'] else None,
        'director': r_data['movieInfoResult']['movieInfo']['directors'][0].get('peopleNm', None) if r_data['movieInfoResult']['movieInfo']['directors'] else None,
        'actor': ', '.join(actor['peopleNm'] for actor in r_data['movieInfoResult']['movieInfo']['actors']) if r_data['movieInfoResult']['movieInfo']['actors'] else None,
        'showTypeGroupNm': ', '.join(stype['showTypeGroupNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
        'showTypeNm': ', '.join(stype['showTypeNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
        'audits': ', '.join(audit['watchGradeNm'] for audit in r_data['movieInfoResult']['movieInfo'].get('audits', [])) if r_data['movieInfoResult']['movieInfo'].get('audits') else None,
        
    }

    info_df = pd.DataFrame([selected_data])
    info_dfs.append(info_df)


total_info_df = pd.concat(info_dfs, ignore_index=True)
total_info_df.head()


,movieCd,movieNm,showTm,prdtYear,openDt,prdtStatNm,typeNm,nationNm,genreNm,director,actor,showTypeGroupNm,showTypeNm,audits
0,20141111,"님아, 그 강을 건너지 마오",85,2014,20141127,개봉,장편,한국,다큐멘터리,진모영,"조병만, 강계열","필름, 2D, 2D","필름, 디지털, 디지털 가치봄",전체관람가
1,20130574,개를 훔치는 완벽한 방법,109,2014,20141231,개봉,장편,한국,드라마,김성호,"김혜자, 이레, 최민수, 강혜정, 이천희, 이지원, 홍은택, 김도엽, 김재화, 이기...","필름, 2D, 2D","필름, 디지털, 디지털 가치봄",전체관람가


In [23]:
%%time
import requests

key = '75b69dc273dc0e8cef4e9cfb4736799f'
movie_codes = df[df['current_date'].dt.year >= 2024]['movieCd'].drop_duplicates().tolist()

info_dfs = []

for movieCd in movie_codes:
    try:
        url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
        response = requests.get(url)
        r_data = response.json()

        # 필요한 필드 선택
        selected_data = {
            'movieCd': r_data['movieInfoResult']['movieInfo'].get('movieCd', None),
            'movieNm': r_data['movieInfoResult']['movieInfo'].get('movieNm', None),
            'showTm': r_data['movieInfoResult']['movieInfo'].get('showTm', None),
            'prdtYear': r_data['movieInfoResult']['movieInfo'].get('prdtYear', None),
            'openDt': r_data['movieInfoResult']['movieInfo'].get('openDt', None),
            'prdtStatNm': r_data['movieInfoResult']['movieInfo'].get('prdtStatNm', None),
            'typeNm': r_data['movieInfoResult']['movieInfo'].get('typeNm', None),
            'nationNm': r_data['movieInfoResult']['movieInfo']['nations'][0]['nationNm'] if r_data['movieInfoResult']['movieInfo'].get('nations') else None,
            'genreNm': r_data['movieInfoResult']['movieInfo']['genres'][0]['genreNm'] if r_data['movieInfoResult']['movieInfo'].get('genres') else None,
            'director': r_data['movieInfoResult']['movieInfo']['directors'][0]['peopleNm'] if r_data['movieInfoResult']['movieInfo'].get('directors') else None,
            'actor': ', '.join(actor['peopleNm'] for actor in r_data['movieInfoResult']['movieInfo'].get('actors', [])),
            'showTypeGroupNm': ', '.join(stype['showTypeGroupNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
            'showTypeNm': ', '.join(stype['showTypeNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
            'audits': ', '.join(audit['watchGradeNm'] for audit in r_data['movieInfoResult']['movieInfo'].get('audits', [])) if r_data['movieInfoResult']['movieInfo'].get('audits') else None
        }

        info_df = pd.DataFrame([selected_data])
        info_dfs.append(info_df)
    except Exception as e:
        print(f"Error occurred while processing movie code {movieCd}: {e}")

total_info_df = pd.concat(info_dfs, ignore_index=True)
total_info_df.head()


CPU times: user 377 ms, sys: 80.7 ms, total: 458 ms
Wall time: 46.8 s


,movieCd,movieNm,showTm,prdtYear,openDt,prdtStatNm,typeNm,nationNm,genreNm,director,actor,showTypeGroupNm,showTypeNm,audits
0,20234114,괴물,126,2023,20231129,개봉,장편,일본,드라마,고레에다 히로카즈,"안도 사쿠라, 다나카 유코, 나가야마 에이타, 쿠로카와 소야, 히이라기 히나타",2D,디지털,12세이상관람가
1,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",68,2023,20231227,개봉,장편,영국,애니메이션,None,,2D,디지털 더빙,전체관람가
2,20236080,류이치 사카모토: 오퍼스,103,2023,20231227,개봉,장편,일본,공연,소라 네오,사카모토 류이치,2D,디지털,전체관람가
3,20233299,크레센도,111,2023,20231220,개봉,장편,미국,다큐멘터리,헤더 윌크,,2D,디지털,전체관람가
4,20235923,사랑은 낙엽을 타고,80,2023,20231220,개봉,장편,핀란드,멜로/로맨스,아키 카우리스마키,,2D,디지털,12세이상관람가


In [24]:
total_info_df.to_csv('../data/boxoffice_info_y_24.csv',index = False, encoding = 'utf-8-sig')

In [25]:
total_info_df = pd.read_csv('../data/boxoffice_info_15_23.csv')
total_info_y_df = pd.read_csv('../data/boxoffice_info_y_24.csv')

In [28]:
total_info_df = pd.concat([total_info_df, total_info_y_df], axis=0)

In [29]:
total_info_df.drop_duplicates(inplace = True)

In [30]:
total = pd.merge(df,total_info_df, on= ['movieCd','movieNm'], how = 'left')

In [31]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33780 entries, 0 to 33779
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rnum             33780 non-null  int64         
 1   rank             33780 non-null  int64         
 2   rankInten        33780 non-null  int64         
 3   rankOldAndNew    33780 non-null  object        
 4   movieCd          33780 non-null  object        
 5   movieNm          33780 non-null  object        
 6   openDt_x         33780 non-null  object        
 7   salesAmt         33780 non-null  int64         
 8   salesShare       33780 non-null  float64       
 9   salesInten       33780 non-null  int64         
 10  salesChange      33780 non-null  float64       
 11  salesAcc         33780 non-null  int64         
 12  audiCnt          33780 non-null  int64         
 13  audiInten        33780 non-null  int64         
 14  audiChange       33780 non-null  float

In [32]:
total[['openDt_x','openDt_y']] # 년도별 형식이 다름

,openDt_x,openDt_y
0,2014-11-27,20141127.0
1,2014-12-31,20141231.0
2,2014-12-24,20141224.0
3,2014-12-31,NaN
4,2015-01-01,NaN
...,...,...
33775,2017-07-13,20170713.0
33776,2024-03-13,20240313.0
33777,1987-05-23,19870523.0
33778,2024-03-21,20240321.0


In [33]:
# 날짜 형식으로 변환
total['openDt_y'] = pd.to_datetime(total['openDt_y'], format='%Y%m%d')

In [34]:
total['openDt_x'].fillna(total['openDt_y'], inplace=True)

In [35]:
total.drop(columns = 'openDt_y', inplace = True)
total.rename(columns = {'openDt_x' : 'openDt'}, inplace = True)

In [36]:
total.openDt

0        2014-11-27
1        2014-12-31
2        2014-12-24
3        2014-12-31
4        2015-01-01
            ...    
33775    2017-07-13
33776    2024-03-13
33777    1987-05-23
33778    2024-03-21
33779    2024-02-07
Name: openDt, Length: 33780, dtype: object

In [37]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33780 entries, 0 to 33779
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rnum             33780 non-null  int64         
 1   rank             33780 non-null  int64         
 2   rankInten        33780 non-null  int64         
 3   rankOldAndNew    33780 non-null  object        
 4   movieCd          33780 non-null  object        
 5   movieNm          33780 non-null  object        
 6   openDt           33780 non-null  object        
 7   salesAmt         33780 non-null  int64         
 8   salesShare       33780 non-null  float64       
 9   salesInten       33780 non-null  int64         
 10  salesChange      33780 non-null  float64       
 11  salesAcc         33780 non-null  int64         
 12  audiCnt          33780 non-null  int64         
 13  audiInten        33780 non-null  int64         
 14  audiChange       33780 non-null  float

In [38]:
total.current_date

0       2015-01-01
1       2015-01-01
2       2015-01-01
3       2015-01-01
4       2015-01-01
           ...    
33775   2024-03-31
33776   2024-03-31
33777   2024-03-31
33778   2024-03-31
33779   2024-03-31
Name: current_date, Length: 33780, dtype: datetime64[ns]

In [39]:
total.to_csv('../data/boxoffice_y_15_24_total.csv',index = False, encoding = 'utf-8-sig')